# Data Wrangling or Data Munging

Here we will concentrate on the following sub-sections of this methodology:

1> Data collection: To understand different data retrieval mechanisms for 
                    different data types. -> in very brief

2> Data description: To understand various attributes and properties of the
                     data collected. -> in very brief

3> Data wrangling: To prepare data for consumption in the modeling steps.

4> Data visualization: To visualize different attributes for sharing results, better understanding, and so on.  -> "Covered through matplotlib"

In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

from IPython.display import display

pd.options.mode.chained_assignment = None
import warnings; warnings.simplefilter('ignore')  # to suppress warnings

## Utilities

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """
    
    # sentinels
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

## Generate a Sample Dataset

In [3]:
df = generate_sample_data(row_count=1000)
df.head(10)

,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,22,NaN,n
1,1001,NaN,5887,554,39,4347.19,n
2,1002,2016-01-02,5276,297,11,1375.57,n
3,1003,NaN,5643,988,2,1127.97,n
4,1004,NaN,5004,439,30,2487.95,n
5,1005,2016-01-25,5847,423,23,654.73,n
6,1006,NaN,5242,458,2,3324.90,n
7,1007,NaN,5239,666,25,1865.62,n
8,1008,2016-01-22,5855,889,33,3225.06,n
9,1009,NaN,5065,677,5,6081.99,n


In [9]:
df.shape
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 969 non-null float64
User Type             1000 non-null object
dtypes: float64(1), int32(2), int64(2), object(2)
memory usage: 47.0+ KB
None
         Serial No      User ID   Product ID  Quantity Purchased        Price
count  1000.000000  1000.000000  1000.000000         1000.000000   969.000000
mean   1452.154000  5486.266000   557.567000           20.747000  2193.544066
std     386.403105   335.131963   283.116489           11.594543  1555.087434
min      -1.000000  -101.000000     0.000000            1.000000     0.690000
25%    1221.750000  5257.000000   294.250000           11.000000   971.100000
50%    1483.000000  5489.000000   526.000000         

### Describe the Dataset

In [11]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 61


Sample Indices with missing data:: [0, 1, 3, 4, 6]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 969 non-null float64


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,22,NaN,n
1,1001,NaN,5887,554,39,4347.19,n
2,1002,2016-01-02,5276,297,11,1375.57,n
3,1003,NaN,5643,988,2,1127.97,n
4,1004,NaN,5004,439,30,2487.95,n


### Rename Columns

In [15]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


In [16]:
cleanup_column_names(df)

In [17]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


In [18]:
df.head()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,1000,NaN,-101,0,22,NaN,n
1,1001,NaN,5887,554,39,4347.19,n
2,1002,2016-01-02,5276,297,11,1375.57,n
3,1003,NaN,5643,988,2,1127.97,n
4,1004,NaN,5004,439,30,2487.95,n


In [12]:
print('{0} and {1}'.format('Geeks', 'Portal')) 
  
print('{1} and {0}'.format('Geeks', 'Portal')) 

Geeks and Portal
Portal and Geeks


### Sort Rows on defined attributes

In [19]:
display(df.sort_values(['serial_no','price'],ascending=[True,False]).head())
#display(df.sort_values(['serial_no','price'],ascending=[True,False]).tail())

# first sorted on serial_no, all products having same serial_no sorted on price

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
643,-1,2016-01-31,5434,367,20,5761.43,c
326,-1,2016-01-17,5466,883,3,5602.76,c
401,-1,2016-01-27,5887,554,11,4887.37,b
610,-1,2016-01-02,5963,143,23,4397.70,b
55,-1,2016-06-01,5111,502,32,4372.39,n


### Rearrange Columns in a Dataframe

In [20]:
display(df[['serial_no','date','user_id','user_type',
           'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,22,NaN
1,1001,NaN,5887,n,554,39,4347.19
2,1002,2016-01-02,5276,n,297,11,1375.57
3,1003,NaN,5643,n,988,2,1127.97
4,1004,NaN,5004,n,439,30,2487.95


### Filtering Columns

Using Column Index

In [11]:
df.head()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,-1,NaN,-101,0,30,NaN,n
1,1001,NaN,5801,650,32,218.24,n
2,1002,2016-07-02,5220,318,17,135.59,n
3,-1,2016-01-02,5609,658,27,NaN,n
4,1004,NaN,5030,705,9,2545.91,n


In [23]:
# Exercise Cell, Not a part of NB
# importing pandas as pd 
import pandas as pd 
  
# Creating the DataFrame 
temp_df = pd.DataFrame({'Weight':[45, 88, 56, 15, 71], 
                   'Name':['Sam', 'Andrea', 'Alex', 'Robin', 'Kia'], 
                   'Age':[14, 25, 55, 8, 21]}) 
  
# Print the DataFrame 
# type your code here
print(temp_df)

# return the numpy representation of  
# this dataframe 
print("-------------------------")
# Print the result 
# type your code here


print("-------------------------")
# Print the result 
print(temp_df['Weight'].values)

print("-------------------------")
# Print the result 
# type your code here
print(temp_df['Name'].values)

print("-------------------------")
# Print the result 
print(temp_df['Age'].values)

print("-------------------------")
# Print the result 
print(temp_df['Name'].values[0:3])

print("-------------------------")
# Print the result 
# type your code here
print(temp_df.iloc[:,1].values[0:3])


   Weight    Name  Age
0      45     Sam   14
1      88  Andrea   25
2      56    Alex   55
3      15   Robin    8
4      71     Kia   21
-------------------------
-------------------------
[45 88 56 15 71]
-------------------------
['Sam' 'Andrea' 'Alex' 'Robin' 'Kia']
-------------------------
[14 25 55  8 21]
-------------------------
['Sam' 'Andrea' 'Alex']
-------------------------
['Sam' 'Andrea' 'Alex']


In [25]:
# type your code here
df.head(10)

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,1000,NaN,-101,0,22,NaN,n
1,1001,NaN,5887,554,39,4347.19,n
2,1002,2016-01-02,5276,297,11,1375.57,n
3,1003,NaN,5643,988,2,1127.97,n
4,1004,NaN,5004,439,30,2487.95,n
5,1005,2016-01-25,5847,423,23,654.73,n
6,1006,NaN,5242,458,2,3324.90,n
7,1007,NaN,5239,666,25,1865.62,n
8,1008,2016-01-22,5855,889,33,3225.06,n
9,1009,NaN,5065,677,5,6081.99,n


In [26]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])
# type your comment here
#df.iloc[row,col]......:means we dont wish to specify any value
#values[0:10] means 0 to 10


[  0 554 297 988 439 423 458 666 889 677]


In [15]:
# print 5 values from column at index 3
print(df.iloc[:,3].values[0:5])

[  0 650 318 658 705]


Using Column Name

In [27]:
# print 10 values of quantity purchased
# type your code here
print(df.quantity_purchased.values[0:10])

[22 39 11  2 30 23  2 25 33  5]


In [17]:
df.head()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
0,-1,NaN,-101,0,30,NaN,n
1,1001,NaN,5801,650,32,218.24,n
2,1002,2016-07-02,5220,318,17,135.59,n
3,-1,2016-01-02,5609,658,27,NaN,n
4,1004,NaN,5030,705,9,2545.91,n


Using Column Datatype

In [29]:
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])
# we are printing only 0th column, i.e price

[    nan 4347.19 1375.57 1127.97 2487.95  654.73 3324.9  1865.62 3225.06
 6081.99]


In [19]:
# type your code here



[-101 5801 5220 5609 5030 5283 5173 5077 5533 5279]


In [20]:
print(df.select_dtypes(include=['int64']).values[:10,1]) #product_id

[   0  650  318  658  705 1003  440  395  968 1081]


In [33]:
print(df.select_dtypes(include=['float64','int64']).values[:10,0])
print(df.select_dtypes(include=['float64','int64']).values[:10,1])
print(df.select_dtypes(include=['float64','int64']).values[:10,2])
#print(df.select_dtypes(include=['float64','int64']).values[:10,3]) gives error



[-101. 5887. 5276. 5643. 5004. 5847. 5242. 5239. 5855. 5065.]
[  0. 554. 297. 988. 439. 423. 458. 666. 889. 677.]
[    nan 4347.19 1375.57 1127.97 2487.95  654.73 3324.9  1865.62 3225.06
 6081.99]


### Filtering Rows

Select specific rows

In [34]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-04-02,5963,143,30,1968.36,n
501,1501,2016-05-02,5887,554,16,237.97,a
20,1020,2016-06-01,5815,799,14,950.88,n


Exclude Specific Row indices

In [22]:
display(df.drop([0,2,5], axis=0).head()) 
# type your comments here
# notes: axis=1 would give error as , indexes 0,2,5 don't appear column wise
#default axis is 0
# display(df.drop(['serial_no','user_id'],axis=1).head())



,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5801,650,32,218.24,n
3,-1,2016-01-02,5609,658,27,NaN,n
4,1004,NaN,5030,705,9,2545.91,n
6,1006,NaN,5173,440,8,36.37,n
7,1007,NaN,5077,395,2,125.17,n


Conditional Filtering

In [35]:
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5887,554,39,4347.19,n
4,1004,NaN,5004,439,30,2487.95,n
8,1008,2016-01-22,5855,889,33,3225.06,n
10,1010,2016-04-02,5963,143,30,1968.36,n
13,1013,2016-01-26,5137,357,27,1537.71,d


In [37]:
# type your code here
display(df[df.price>25].head())


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5887,554,39,4347.19,n
2,1002,2016-01-02,5276,297,11,1375.57,n
3,1003,NaN,5643,988,2,1127.97,n
4,1004,NaN,5004,439,30,2487.95,n
5,1005,2016-01-25,5847,423,23,654.73,n


Offset from top of the dataframe

In [25]:
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-01-28,5584,405,20,2570.49,c
101,1101,2016-01-13,5801,650,7,5606.11,c
102,1102,2016-01-27,5220,318,15,3935.87,c
103,1103,2016-01-28,5609,658,31,3909.88,b
104,1104,2016-07-01,5030,705,12,2360.86,d


Offset from bottom of the dataframe

In [26]:
display(df[-10:].head()) # type your comment here


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-01-26,5644,798,27,876.57,c
991,1991,2016-07-02,5800,962,37,3017.64,d
992,1992,2016-01-24,5653,218,36,3829.08,b
993,1993,2016-06-02,5905,1083,41,2187.42,d
994,1994,2016-01-23,5478,116,11,2222.88,d


### TypeCasting/Data Type Conversion

In [71]:
# Exercise Cell, not a part of NB
# importing pandas as pd 
import pandas as pd 

# Creating the dataframe 
d_df = pd.DataFrame({'Date':['11/8/2011', '04/23/2008', '10/2/2019'], 
                    'Event':['Music', 'Poetry', 'Theatre'], 
                    'Cost':[10000, 5000, 15000]}) 

# Print the dataframe 
print(d_df) 

print("--------------------")
# Now we will check the data type 
# of the 'Date' column 
# type your code here
d_df.info()

         Date    Event   Cost
0   11/8/2011    Music  10000
1  04/23/2008   Poetry   5000
2   10/2/2019  Theatre  15000
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
Date     3 non-null object
Event    3 non-null object
Cost     3 non-null int64
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes


In [72]:
# Exercise Cell, not a part of NB
# convert the 'Date' column to datetime format 
d_df['Date']= pd.to_datetime(d_df['Date']) 

# Check the format of 'Date' column 
d_df.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
Date     3 non-null datetime64[ns]
Event    3 non-null object
Cost     3 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 152.0+ bytes


In [73]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
# type your code here
print(d_df.dtypes)

Date     datetime64[ns]
Event            object
Cost              int64
dtype: object


### Apply/Map Usage

Map : Create a derived attribute using map. map() works element wise.

In [74]:
df.tail()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type
995,1995,2016-01-26,5012,379,6,3777.26,d,3.0
996,1996,2016-11-02,5449,122,39,2245.56,b,1.0
997,1997,2016-02-01,5350,435,4,1784.80,d,3.0
998,1998,2016-09-01,5557,155,12,1682.16,d,3.0
999,1999,2016-11-02,5406,755,28,1085.25,b,1.0


In [75]:
df['user_class'] = df['user_type'].map(expand_user_type)
# map function applies the user defn method expand_user_type to each value of 
# user_type
display(df.tail())

# -- expand_user_type function is defn in the Utilities. 
# -- Its body is shown here just for reference.
#  def expand_user_type(u_type):
#   if u_type in ['a','b']:
#   return 'new'
#   elif u_type == 'c':
#   return 'existing'
#   elif u_type == 'd':
#   return 'loyal_existing'
#   else:
#   return 'error'

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class
995,1995,2016-01-26,5012,379,6,3777.26,d,3.0,loyal_existing
996,1996,2016-11-02,5449,122,39,2245.56,b,1.0,new
997,1997,2016-02-01,5350,435,4,1784.80,d,3.0,loyal_existing
998,1998,2016-09-01,5557,155,12,1682.16,d,3.0,loyal_existing
999,1999,2016-11-02,5406,755,28,1085.25,b,1.0,new


Markdown for Jupyter Notebook Cheatsheet
--
https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed

Apply: Using apply to get attribute ranges

In [76]:
# The apply() function is used to perform actions on the whole object, 
# depending upon the axis (default is on all rows).
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

# type your code here
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

display(df.select_dtypes(include=[np.number]).apply(lambda x: x.mean()))


serial_no             2000.00
user_id               6099.00
product_id            1090.00
quantity_purchased      40.00
price                 8206.79
encoded_user_type        3.00
dtype: float64

serial_no             2000.00
user_id               6099.00
product_id            1090.00
quantity_purchased      40.00
price                 8206.79
encoded_user_type        3.00
dtype: float64

serial_no             1452.154000
user_id               5486.266000
product_id             557.567000
quantity_purchased      20.747000
price                 2193.544066
encoded_user_type        1.492784
dtype: float64

Applymap: Extract week from date

In [77]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

# lambda fn gets the week of the transaction from the date attribute

In [78]:
df.head()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
0,1000,NaT,-101,0,22,NaN,n,NaN,error,0
1,1001,NaT,5887,554,39,4347.19,n,NaN,error,0
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
3,1003,NaT,5643,988,2,1127.97,n,NaN,error,0
4,1004,NaT,5004,439,30,2487.95,n,NaN,error,0


In [79]:
#to print week from date
df['date'].dt.week
#df.date.dt.week

0       NaN
1       NaN
2      53.0
3       NaN
4       NaN
5       4.0
6       NaN
7       NaN
8       3.0
9       NaN
10     13.0
11      NaN
12      3.0
13      4.0
14      3.0
15      3.0
16      4.0
17     22.0
18     22.0
19      NaN
20     22.0
21     22.0
22      NaN
23     39.0
24     53.0
25      NaN
26      4.0
27      NaN
28      5.0
29      2.0
       ... 
970    26.0
971    48.0
972    44.0
973    39.0
974     3.0
975    17.0
976     2.0
977    39.0
978    13.0
979    35.0
980     3.0
981    44.0
982    26.0
983    22.0
984     9.0
985     9.0
986     5.0
987     2.0
988     4.0
989     4.0
990     3.0
991    22.0
992     2.0
993    53.0
994     9.0
995     4.0
996    44.0
997     5.0
998    35.0
999    44.0
Name: date, Length: 1000, dtype: float64

In [80]:
 display(df.head()) # display is defined under utilities   


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
0,1000,NaT,-101,0,22,NaN,n,NaN,error,0
1,1001,NaT,5887,554,39,4347.19,n,NaN,error,0
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
3,1003,NaT,5643,988,2,1127.97,n,NaN,error,0
4,1004,NaT,5004,439,30,2487.95,n,NaN,error,0


### Missing Values

Imputing Missing Values : Missing values can lead to all sorts of problems when dealing with Machine Learning and Data Science related use cases. Not only can they cause problems for algorithms, they can mess up calculations and even final outcomes. 

Missing values also pose risk of being interpreted in non-standard ways as well leading to confusion and more errors. Hence, imputing missing values carries a lot of weight in the overall data wrangling process.

One of the easiest ways of handling missing values is to ignore or remove them altogether from the dataset. When the dataset is fairly large and we have enough samples of various types required, this option can be safely exercised. We use the dropna() function from pandas in the following snippet to remove rows of data where the date of transaction is missing.

In [81]:
print("Drop Rows with missing dates::" )
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

# # dropna -> drops not available values.i.e naT in this case

Drop Rows with missing dates::


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
5,1005,2016-01-25,5847,423,23,654.73,n,NaN,error,4
8,1008,2016-01-22,5855,889,33,3225.06,n,NaN,error,3
10,1010,2016-04-02,5963,143,30,1968.36,n,NaN,error,13
12,1012,2016-01-18,5305,508,7,5678.89,n,NaN,error,3


Fill Missing Price values with mean price

In [82]:
# Often dropping rows is a very expensive and unfeasible option. 
# In many scenarios, missing values are imputed using the help of other 
# values in the dataframe. One commonly used trick is to replace missing
# values with a central tendency measure like mean or median.
# fillna -> fills 'not available' values
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)
# type your code here
df_dropped['price'].head()

2     1375.57
5      654.73
8     3225.06
10    1968.36
12    5678.89
Name: price, dtype: float64

In [83]:
# type your code here
df_dropped['price'].isnull().sum()

0

Fill Missing user_type values with value from previous row (forward fill) 

In [84]:
print("Fill Missing user_type values with value from previous row (forward fill) ::" )
df_dropped['user_type'].fillna(method='ffill',inplace=True) #if false the updates are rolled back
df_dropped.head()

Fill Missing user_type values with value from previous row (forward fill) ::


,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
5,1005,2016-01-25,5847,423,23,654.73,n,NaN,error,4
8,1008,2016-01-22,5855,889,33,3225.06,n,NaN,error,3
10,1010,2016-04-02,5963,143,30,1968.36,n,NaN,error,13
12,1012,2016-01-18,5305,508,7,5678.89,n,NaN,error,3


Fill Missing user_type values with value from next row (backward fill)

In [85]:
df_dropped['user_type'].fillna(method='bfill',inplace=True) 
# keeping inplace true writes the new data in the data set itself.


inplace=True   i.e like a++
df-> changes copies to original
thereby new data space not consumed.

------

inplace=False i.e like a=a+1
df-> changes not copied to original
thereby ne data space created.

//here do this kind of coding
dfNew = df.fillna(...,inplace=False)

### Duplicates

Drop Duplicate serial_no rows

In [86]:
df_dropped.shape

(969, 10)

In [87]:
df_dropped.head()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
5,1005,2016-01-25,5847,423,23,654.73,n,NaN,error,4
8,1008,2016-01-22,5855,889,33,3225.06,n,NaN,error,3
10,1010,2016-04-02,5963,143,30,1968.36,n,NaN,error,13
12,1012,2016-01-18,5305,508,7,5678.89,n,NaN,error,3


In [88]:
# sample duplicates
# duplicated is a build-in fn
# Lets display the duplicated serial_no
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
# type your code here
print(df_dropped.shape)

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
80,-1,2016-01-13,5315,525,4,2901.75,a,0.0,new,2
210,-1,2016-04-02,5963,143,7,4035.79,b,1.0,new,13
239,-1,2016-01-20,5463,773,5,3434.85,d,3.0,loyal_existing,3
255,-1,2016-03-01,5111,502,2,194.49,a,0.0,new,9
260,-1,2016-03-01,5006,489,22,3009.69,b,1.0,new,9


(969, 10)


In [89]:
# drop_duplicates is a build-in function 
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [90]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
5,1005,2016-01-25,5847,423,23,654.73,n,NaN,error,4
8,1008,2016-01-22,5855,889,33,3225.06,n,NaN,error,3
10,1010,2016-04-02,5963,143,30,1968.36,n,NaN,error,13
12,1012,2016-01-18,5305,508,7,5678.89,n,NaN,error,3


Shape of df=(939, 10)


Remove rows which have less than 3 attributes with non-missing data

In [91]:
# there are certain conditions where a record is not much of use 
# if it has more than a certain threshold of attribute values missing. 
# For instance, if in our dataset a transaction has less than three
# attributes as non-null, the transaction might almost be unusable. 
# In such a scenario, it might be advisable to drop that data point itself. 
# We can filter out such data points using the function dropna() 
# with the parameter thresh set to the threshold of null attributes

# In short, it removes each row which has more than 3 null values
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
0,1000,NaT,-101,0,22,NaN,n,NaN,error,0
1,1001,NaT,5887,554,39,4347.19,n,NaN,error,0
2,1002,2016-01-02,5276,297,11,1375.57,n,NaN,error,53
3,1003,NaT,5643,988,2,1127.97,n,NaN,error,0
4,1004,NaT,5004,439,30,2487.95,n,NaN,error,0


Shape of df=(1000, 10)


### Encode Categoricals

One Hot Encoding using get_dummies()

In [92]:
# method to convert the categorical variable into indicator variables 
# use the get_dummies() function.
# type your code here
pd.get_dummies(df,columns=['user_type']).head()

,serial_no,date,user_id,product_id,quantity_purchased,price,encoded_user_type,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,22,NaN,NaN,error,0,0,0,0,0,1
1,1001,NaT,5887,554,39,4347.19,NaN,error,0,0,0,0,0,1
2,1002,2016-01-02,5276,297,11,1375.57,NaN,error,53,0,0,0,0,1
3,1003,NaT,5643,988,2,1127.97,NaN,error,0,0,0,0,0,1
4,1004,NaT,5004,439,30,2487.95,NaN,error,0,0,0,0,0,1


Label Mapping

In [93]:
# using the map() function, where we simply map each value 
# from the allowed set to a numeric value
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
# type your code here
df.tail()

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
995,1995,2016-01-26,5012,379,6,3777.26,d,3.0,loyal_existing,4
996,1996,2016-11-02,5449,122,39,2245.56,b,1.0,new,44
997,1997,2016-02-01,5350,435,4,1784.80,d,3.0,loyal_existing,5
998,1998,2016-09-01,5557,155,12,1682.16,d,3.0,loyal_existing,35
999,1999,2016-11-02,5406,755,28,1085.25,b,1.0,new,44


### Random Sampling data from DataFrame

In [94]:
display(df.sample(frac=0.2, replace=True, random_state=42).head()) 
# explaination for replace parameter
# when sampling, the records are removed from the orginal dataset, so that
# their is no repetition of samples in the o/p.
# but if the size of the sample is greater than the dataset itself then keep
# replace = true
"""replace=True is like CBV
replace=False is like CBR"""

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
102,1102,2016-01-02,5276,297,19,699.18,c,2.0,existing,53
435,1435,2016-08-01,5732,746,2,1604.55,a,0.0,new,31
860,1860,2016-09-01,5006,489,4,2649.99,c,2.0,existing,35
270,1270,2016-01-27,5129,943,19,885.48,b,1.0,new,4
106,1106,NaT,5242,458,2,3060.62,b,1.0,new,0


'replace=True is like CBV\nreplace=False is like CBR'

### Normalizing Numeric Values

Attribute normalization is the process of standardizing the range of values of attributes. Machine learning algorithms in many cases utilize distance metrics, attributes or features of different scales/ranges which might adversely affect the calculations or bias the outcomes. Normalization is also called feature scaling.

Normalize price values using  **Min-Max Scaler**

In [95]:
# Visit this link before you proceed
# https://www.geeksforgeeks.org/python-difference-between-pandas-copy-and-copying-through-variables/
df_normalized = df.dropna().copy() 
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))#all scaling functions works on series
df_normalized['price'] = np_scaled.reshape(-1,1)

# reshape(-1,1) works like this :
# -1 means we dont know the no. of rows. hence it would take len(df).
# the 2nd parameter being 1 -> means I want only 1 column
# ex: assume z is 2D numpy array and z.shape is (3,4) 
# z.reshape(-1) would give you a 1D np.array
# now z.shape would be (12,)  like 
# array([1,2,3,4,5,6,7,8,9,10,11,12])
# and reshaping it as z.reshape(-1,1) would give us (12,1). 
# i.e 1 column with all row values. 

In [96]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
13,1013,2016-01-26,5137,357,27,0.187286,d,3.0,loyal_existing,4
15,1015,2016-01-18,5573,600,26,0.727886,d,3.0,loyal_existing,3
17,1017,2016-06-02,5257,857,15,0.246512,d,3.0,loyal_existing,22
18,1018,2016-06-02,5738,656,37,0.192766,d,3.0,loyal_existing,22
23,1023,2016-10-02,5524,864,14,0.407119,b,1.0,new,39


Normalize quantity purchased values using  **Robust Scaler**

In [97]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
# type your code here

df_normalized['quantity_purchased']=rs_scaled.reshape(-1,1)

# interested people may research the maths behind min-max scaler and
# robust_scaler. 

In [98]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,encoded_user_type,user_class,purchase_week
13,1013,2016-01-26,5137,357,0.35,1537.71,d,3.0,loyal_existing,4
15,1015,2016-01-18,5573,600,0.30,5974.30,d,3.0,loyal_existing,3
17,1017,2016-06-02,5257,857,-0.25,2023.76,d,3.0,loyal_existing,22
18,1018,2016-06-02,5738,656,0.85,1582.68,d,3.0,loyal_existing,22
23,1023,2016-10-02,5524,864,-0.30,3341.83,b,1.0,new,39


### Data Summarization

Data summarization refers to the process of preparing a compact representation of raw data at hand. This process involves aggregation of data using different statistical, mathematical, and other methods. Summarization is helpful for visualization, compressing raw data, and better understanding of its attributes.

Condition based aggregation

In [99]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2267.4923504273506


Condtion based counts

In [100]:
print(df['purchase_week'].value_counts())
# counts the number of transactions per week

3     162
4     158
2     125
35     59
5      54
53     53
13     53
26     48
9      42
39     41
22     41
31     40
44     35
0      31
18     25
48     17
17     16
Name: purchase_week, dtype: int64


### Group By

Group By certain attributes

In [101]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())
# This statement generates a tabular output representing 
# sum of quantities purchased by each user_class.

user_class
error              588
existing          5116
loyal_existing    5055
new               9988
Name: quantity_purchased, dtype: int32


Group By with different aggregate functions

In [57]:
# The groupby() function is a powerful interface that allows us 
# to perform complex groupings and aggregations.
# With groupby() we can perform multi-attribute groupings 
# and apply multiple aggregations across attributes.

# variant-1: multiple aggregations on single attribute
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,569,18.354839,31
existing,5100,21.161826,241
loyal_existing,5077,20.722449,245
new,10351,21.430642,483


Group by specific aggregate functions for each attribute

In [58]:
# variant-2: different aggregation functions for each attribute
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2194.714286                  39
existing       c          2241.889700                  41
loyal_existing d          2312.115443                  41
new            a          2275.840491                  41
               b          2273.187814                  41

Group by with multiple agg for each attribute

In [59]:
# Variant 3: Here, we do a combination of variants 1 and 2, 
# i.e., we apply multiple aggregations on the price field while 
# applying only a single one on quantity_purchased. 
# Note : a dictionary is passed, as shown in the snippet.
display(df.groupby(['user_class','user_type']).agg({'price':{  'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            61452.00  2194.714286    1590.653715   31.0   
existing       c           522360.30  2241.889700    1658.993006  241.0   
loyal_existing d           547971.36  2312.115443    1571.826738  245.0   
new            a           509788.27  2275.840491    1491.190211  230.0   
               b           561477.39  2273.187814    1563.262156  253.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        569  
existing       c                       5100  
loyal_existing d                       5077  
new            a                       5024  
               b                       5327

### Pivot Tables

In [60]:
#HW: Remember its same as stack() and unstack()
display(df.groupby(['user_class','user_type']).agg({'price':{  'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

# pivot table shows us comprehensive information of mean price 
# date-wise , user_type wise

price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            61452.00  2194.714286    1590.653715   31.0   
existing       c           522360.30  2241.889700    1658.993006  241.0   
loyal_existing d           547971.36  2312.115443    1571.826738  245.0   
new            a           509788.27  2275.840491    1491.190211  230.0   
               b           561477.39  2273.187814    1563.262156  253.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        569  
existing       c                       5100  
loyal_existing d                       5077  
new            a                       5024  
               b                       5327